# Introduction to PowerShell in Azure

## PowerShell

PowerShell is a powerful scripting and automation framework developed by Microsoft. It is designed for task automation and configuration management and is particularly useful for managing and automating Microsoft Windows environments. PowerShell uses a command-line interface with a scriptable approach, and it's built on the .NET Framework.

* [What is PowerShell?](https://learn.microsoft.com/powershell/scripting/overview?view=powershell-7.3&WT.mc_id=AZ-MVP-5004796)

## PowerShell and Microsoft Azure

When it comes to Microsoft Azure, PowerShell provides a robust set of cmdlets (pronounced "command-lets") that enable you to interact with and manage Azure resources, making it a valuable tool for working with Azure services. 

When you run a PowerShell cmdlet to, for example, create a virtual machine or retrieve information about an Azure resource, the cmdlet translates your request into an HTTP request to the relevant Azure REST API endpoint.

* [Azure PowerShell Documentation](https://learn.microsoft.com/powershell/azure/?view=azps-10.4.1&WT.mc_id=AZ-MVP-5004796)

## Notebook

Computational notebooks are interactive environments that allow users to write and execute code, view results, and add explanatory text in a single document. They are often used for data analysis, scientific research, and sharing code and insights.

The term "polyglot" in this context means that you can use multiple programming languages within the same notebook. For example, Jupyter notebooks, one of the most popular types of computational notebooks, can support various programming languages like Python, R, Julia, and more.

The ability to use multiple languages in a single notebook can be valuable when you want to combine the strengths of different programming languages for a specific task or when collaborating with individuals who have expertise in different languages. It allows you to create a more versatile and flexible computing environment within a single document.

We are using a .NET interactive notebook supporting PowerShell, for the purposes of this course.

* [Polyglot Notebooks! Multi-language notebooks in Visual Studio Code](https://devblogs.microsoft.com/dotnet/announcing-polyglot-notebooks-harness-the-power-of-multilanguage-notebooks-in-visual-studio-code/?WT.mc_id=AZ-MVP-5004796)

In [ ]:
# Get PSVersion
# The command $PSVersionTable is used to display the version of PowerShell that is currently being used.
# The output of this command includes information such as the PowerShell version, the CLR version, the build version, and the PSRemoting protocol version.
# This command is useful when you need to check the PowerShell version for compatibility with scripts or modules, or when reporting issues.
# A supported version of PowerShell version 7 or higher is the recommended version of PowerShell for use with the Az PowerShell module on all platforms including Windows, Linux, and macOS.


$PSVersionTable

## Setting up your Azure environment for PowerShell

In [ ]:
# Install Azure Modules
# The Set-PSRepository cmdlet is used to set values for a registered repository. 
# The Install-Module cmdlet is used to download one or more modules from an online gallery and installs them on the local computer. In this case, the command is installing the Az module, which provides cmdlets for managing Azure resources.
Set-PSRepository -Name 'PSGallery' -InstallationPolicy Trusted
Install-Module Az 

In [ ]:
# Install Azure Module as Current User (as opposed to System)
# '-Scope CurrentUser` specifies that the module should be installed only for the current user. If you don't specify a scope, the default is `AllUsers`, which requires administrator permissions.

Install-Module -Name Az -Repository PSGallery -Scope CurrentUser

In [ ]:
# Install specific version of Azure Modules

Install-Module -Name Az -RequiredVersion 4.6.1 

In [ ]:
# Update Azure Module
# The command Get-InstalledModule -Name Az* | Update-Module is used to update all installed PowerShell modules that start with "Az".
Set-PSRepository -Name 'PSGallery' -InstallationPolicy Trusted
Get-InstalledModule -Name Az* | Update-Module

In [ ]:
# Gets Az Commands
# Note: Will take a while for all the cmdlets to list.
Get-Command -Noun Az*


In [ ]:
# Get Help
Get-Help -Name Get-AzVM -Examples

## Getting started with Azure PowerShell module

In [ ]:
# Connect to Azure
# WARNING: Interactive authentication is not supported in this session, please run cmdlet 'Connect-AzAccount -UseDeviceAuthentication'.
Connect-AzAccount -UseDeviceAuthentication

In [ ]:
# Connect to Azure using Service Principal authentication

$SecurePassword = ConvertTo-SecureString -String "Password123!" -AsPlainText -Force
$TenantId = 'yyyyyyyy-yyyy-yyyy-yyyy-yyyyyyy'
$ApplicationId = 'zzzzzzzz-zzzz-zzzz-zzzz-zzzzzzzz'
$Credential = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList $ApplicationId, $SecurePassword
Connect-AzAccount -ServicePrincipal -TenantId $TenantId -Credential $Credential

In [ ]:
# Get Azure Subscriptions
Get-AzSubscription

In [ ]:
# Select Azure Subscription
$subid = ''
Set-AzContext -SubscriptionId $subid

In [ ]:
# Get Azure resource groups and resources
Get-AzResourceGroup | Format-Table
Get-AzResource | Format-Table

In [ ]:
# Get Azure resource
Get-AzResource

In [ ]:
# Get Azure resource by ResourceType
Get-AzResource | Where-Object {$_.ResourceType -eq 'Microsoft.Network/virtualNetworks'} 


In [ ]:
# Sort Azure resource by Name and Resource Group
Get-AzResource | Where-Object {$_.ResourceType -eq 'Microsoft.Storage/storageAccounts'} | Sort-Object Name
Get-AzResource | Sort-Object ResourceGroupName 

In [ ]:
# Working with Variables

# Working with variables and data types in PowerShell
$resourceType = 'Microsoft.Network/virtualNetworks'
Get-AzResource | Where-Object {$_.ResourceType -eq $resourceType}

In [ ]:
# Using PowerShell operators for comparisons and calculations

$resources = Get-AzResource
$count = $resources.Count
Write-Host "You have $count resources in your Azure subscription."

In [ ]:
# Scripting constructs: loops and conditional statements
$resources = Get-AzResource

foreach ($resource in $resources) {
    if ($resource.ResourceType -eq 'Microsoft.Network/virtualNetworks') {
        Write-Host "Found a virtual network: $($resource.Name)"
        Write-Host "This virtual network is in $($resource.ResourceGroupName)" -ForegroundColor Green
    }
}

In [ ]:

# Scripting constructs: loops and conditional statements
$subscriptions = Get-AzSubscription

foreach ($subscription in $subscriptions) {
$resource = Get-AzResource | Where-Object {$_.ResourceType -eq 'Microsoft.Network/virtualNetworks'} 

    if ($resource.ResourceType -eq 'Microsoft.Network/virtualNetworks') {
        Write-Host "Found a virtual network: $($resource.Name)" -BackgroundColor Black -ForegroundColor White
        Write-Host "This virtual network is in $($resource.ResourceGroupName)" -ForegroundColor Green
        Write-Host "This Virtual Network is in $($subscription.Name)" -ForegroundColor Green
    }
}

In [ ]:
# Error handling in PowerShell
try {
    Get-AzResource -ResourceGroupName "NonexistentResourceGroup" -ErrorAction Stop
} catch {
    Write-Host "An error occurred: $_. Make sure you have selected the right Resource Group" -ForegroundColor Red
}